# **MACHINE LEARNING**


**1.** Apresentação de um problema real que pode ser resolvido com
modelo supervisionado (2~3 slides contendo o tipo de problema e a
técnica/modelo que resolveria);

    Resposta : A possibilidade de evasão de um aluno de um curso de graduação. 
    Modelo   : Classificação (Regressão  Logística).

**2.** Apresentação de um problema real que pode ser resolvido com
modelo não-supervisionado (2~3 slides contendo o tipo de problema
e a técnica/modelo que resolveria);

    Resposta : Segmentação de carreira de alunos com base em performance em cada área de atuação escolar. 
    Modelo   : Clustering (K-Means).

**3.** Apresentação de um problema real que pode ser resolvido usando ML,
mas não precisa de fato;

    Resposta : Detectar possível hacker em uma rede interna com base no seu comportamento. O mesmo pode ser verificado com base em IPs ou portas acessadas pelo usuário, colocando as condições de identificação ou bloqueio via Firewall.
    Modelo   : Classificação (Regressão  Logística) ou Deep Learning.

**4.** Apresentar um problema implementado (com código), que pode ser:

    a. Um artigo científico que usou algum modelo visto na disciplina
    para resolver um determinado problema. 
    b. Um problema de alguma competição (e.g. Kaggle) que usou
    algum modelo visto na disciplina.
    c. Escolher qualquer problema que o grupo queira (da empresa
    que trabalha, StartupOne) e criar uma aplicação de ML. 

## Modelo de Previsão de Partidas da Copa do Mundo

O objetivo do modelo é prever o resultado de jogos da Copa do Mundo de futebol de 2022 no Quatar.
O intuito do projeto é utilizar os modelos supervisionados de classificação para obter o resultado esperado de cada partida da competição, chegando no possível ganhador.

In [39]:
# Carregamos as principais bibliotecas necessárias
import pandas as pd
import numpy as np

In [40]:
# Realizamos a leitura do dataset
dados = pd.read_csv("results.csv")

In [41]:
# Fazemos a descrição dos dados:
dados.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


In [42]:
# verificamos os tipos de dados
# verificamos se temos dados vazios que precisam ser tratados
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44060 entries, 0 to 44059
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        44060 non-null  object 
 1   home_team   44060 non-null  object 
 2   away_team   44060 non-null  object 
 3   home_score  44059 non-null  float64
 4   away_score  44059 non-null  float64
 5   tournament  44060 non-null  object 
 6   city        44060 non-null  object 
 7   country     44060 non-null  object 
 8   neutral     44060 non-null  bool   
dtypes: bool(1), float64(2), object(6)
memory usage: 2.7+ MB


In [43]:
# Removemos os NAs presentes
dados = dados.dropna();

# Covertemos os placares para inteiros
dados["home_score"] = dados["home_score"].astype("int")
dados["away_score"] = dados["away_score"].astype("int")

dados[["home_score","away_score"]]

,home_score,away_score
0,0,0
1,4,2
2,2,1
3,2,2
4,3,0
...,...,...
44054,1,1
44055,0,2
44056,1,1
44057,5,1


In [44]:
# Separamos os valores de data em colunas diferentes
dados[["year", "month", "day"]] = dados["date"].str.split("-", expand = True)

# Convertemos as strings para inteiros
dados[["year", "month", "day"]] = dados[["year", "month", "day"]].astype("int")
dados[["year", "month", "day"]]

dados.drop("date", axis=1)

,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,month,day
0,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,1872,11,30
1,England,Scotland,4,2,Friendly,London,England,False,1873,3,8
2,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1874,3,7
3,England,Scotland,2,2,Friendly,London,England,False,1875,3,6
4,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,1876,3,4
...,...,...,...,...,...,...,...,...,...,...,...
44054,Albania,Iceland,1,1,UEFA Nations League,Tirana,Albania,False,2022,9,27
44055,Norway,Serbia,0,2,UEFA Nations League,Oslo,Norway,False,2022,9,27
44056,Sweden,Slovenia,1,1,UEFA Nations League,Stockholm,Sweden,False,2022,9,27
44057,Kosovo,Cyprus,5,1,UEFA Nations League,Pristina,Kosovo,False,2022,9,27


In [50]:
# Mudamos os valores de cada coluna categorica para o tipo category, para futuramente transformá-los em números
dados["home_team"]  = dados["home_team"].astype('category')
dados["away_team"]  = dados["away_team"].astype('category')
dados["tournament"] = dados["tournament"].astype('category')
dados["city"]       = dados["city"].astype('category')
dados["country"]    = dados["country"].astype('category')
dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44059 entries, 0 to 44058
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   home_team    44059 non-null  category
 1   away_team    44059 non-null  category
 2   home_score   44059 non-null  int32   
 3   away_score   44059 non-null  int32   
 4   tournament   44059 non-null  category
 5   city         44059 non-null  category
 6   country      44059 non-null  category
 7   neutral      44059 non-null  int32   
 8   year         44059 non-null  int32   
 9   month        44059 non-null  int32   
 10  day          44059 non-null  int32   
 11  winner_home  44059 non-null  int32   
dtypes: category(5), int32(7)
memory usage: 2.0 MB


In [51]:
# Convertemos as colunas categoricas para inteiros
dados["home_team"] = dados["home_team"].cat.codes
dados["away_team"] = dados["away_team"].cat.codes
dados["tournament"] = dados["tournament"].cat.codes
dados["city"] = dados["city"].cat.codes
dados["country"] = dados["country"].cat.codes
dados.head()

,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,month,day,winner_home
0,238,83,0,0,70,642,204,0,1872,11,30,0
1,83,233,4,2,70,1003,68,0,1873,3,8,1
2,238,83,2,1,70,642,204,0,1874,3,7,1
3,83,233,2,2,70,1003,68,0,1875,3,6,0
4,238,83,3,0,70,642,204,0,1876,3,4,1


In [83]:
# Criamos uma coluna para mencionar o vencedor da partida

def classificar_vencedor(x):
    x.head()    
    if x["home_score"] == x["away_score"]:
        x["winner_home"] = 2
    else:
        if x["home_score"] > x["away_score"]:
            x["winner_home"] = 1
        else:
            x["winner_home"] = 0
    return x


dados.apply(lambda x: classificar_vencedor(x),axis=1)

#Converter para inteiro o resultado 

dados["winner_home"] = dados["winner_home"].astype(int)

# Verificar os tipos de valores na neutral
dados["neutral"].unique()

# Transformar em inteiro

dados["neutral"] = dados["neutral"].astype(int)

# Remover coluna de data

dados.drop(["date"], axis=1,inplace=True)

dados.head()



,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,month,day,winner_home
0,238,83,0,0,70,642,204,0,1872,11,30,2
1,83,233,4,2,70,1003,68,0,1873,3,8,1
2,238,83,2,1,70,642,204,0,1874,3,7,1
3,83,233,2,2,70,1003,68,0,1875,3,6,2
4,238,83,3,0,70,642,204,0,1876,3,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
44054,2,122,1,1,127,1776,1,0,2022,9,27,2
44055,197,235,0,2,127,1294,174,0,2022,9,27,0
44056,260,242,1,1,127,1690,225,0,2022,9,27,2
44057,143,70,5,1,127,1415,120,0,2022,9,27,1


In [52]:
# Verificar se está tudo correto
dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44059 entries, 0 to 44058
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   home_team    44059 non-null  int16
 1   away_team    44059 non-null  int16
 2   home_score   44059 non-null  int32
 3   away_score   44059 non-null  int32
 4   tournament   44059 non-null  int16
 5   city         44059 non-null  int16
 6   country      44059 non-null  int16
 7   neutral      44059 non-null  int32
 8   year         44059 non-null  int32
 9   month        44059 non-null  int32
 10  day          44059 non-null  int32
 11  winner_home  44059 non-null  int32
dtypes: int16(5), int32(7)
memory usage: 1.9 MB


In [61]:
# Vamos separar a base em duas partes, uma de treino e outra de teste
from sklearn.model_selection import train_test_split

X = dados.drop(["home_score","away_score","winner_home"], axis=1)
y = dados[["winner_home"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [68]:

# Criar o modelo de regresão 
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, y_train) * 100, 2)
acc_log


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


51.38

In [69]:
# testar a predição 

logreg.predict(np.array([[238,83,70,642,204,0,1872,11,30]]))

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([1])